## Chapter 2-6, 2강 임베딩 기반 유사도 검색 & 미니 RAG (CPU)

- 목표: 문서 임베딩으로 top‑k 검색 구현, 간단 템플릿 요약으로 미니 RAG 구성
- 데이터: 짧은 문서 코퍼스(뉴스 제목/요약), 2k 정도 샘플


### 구성 (Overview)
- 코퍼스 로드/정제 → SBERT 임베딩 생성/캐시 → 최근접 탐색(코사인) → 미니 RAG 응답 조합 → 간단 데모


### 0. 환경 설정 및 라이브러리


In [ ]:
# -*- coding: utf-8 -*-
import os, time, random, numpy as np, pandas as pd, matplotlib.pyplot as plt
from typing import Tuple

plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

_HAS_SBERT = True
try:
    from sentence_transformers import SentenceTransformer, util
except Exception:
    _HAS_SBERT = False

from sklearn.neighbors import NearestNeighbors

# 시드 고정
def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed)
set_seed(42)

print('SBERT 사용 가능:', _HAS_SBERT)


### 1. 코퍼스 준비
- `datasets`에서 뉴스 샘플 로드(없으면 간단 더미 데이터)
- 텍스트 길이 필터, 중복 제거


In [ ]:
_HAS_HF = True
try:
    from datasets import load_dataset
except Exception:
    _HAS_HF = False


def load_news_corpus(n_per_class: int = 500) -> pd.DataFrame:
    if not _HAS_HF:
        # 더미 코퍼스
        titles = [
            'Central bank raises rates to curb inflation',
            'Local team wins championship after dramatic final',
            'Tech company unveils new AI-powered device',
            'Scientists discover exoplanet with Earth-like features',
        ] * n_per_class
        return pd.DataFrame({'title': titles, 'text': titles})
    ds = load_dataset('ag_news', split='train')
    rows = []
    for lab in range(4):
        sub = ds.filter(lambda ex: ex['label']==lab).select(range(n_per_class))
        for r in sub:
            rows.append({'title': r['text'][:120], 'text': r['text']})
    df = pd.DataFrame(rows)
    # 간단 정제: 너무 짧은/긴 텍스트 제거, 중복 제거
    df['len'] = df['text'].str.len()
    df = df[(df['len']>=30) & (df['len']<=500)].drop_duplicates('text').drop(columns=['len']).reset_index(drop=True)
    return df

corpus = load_news_corpus(300)
print(corpus.shape)
corpus.head(2)


### 2. 임베딩 생성 및 캐시(npz)
- 소형 SBERT 모델 사용, CPU 배치 인코딩


In [ ]:
def embed_corpus(texts: list[str], model_name='sentence-transformers/all-MiniLM-L6-v2', batch_size=64):
    if not _HAS_SBERT:
        raise ImportError('sentence-transformers 필요')
    model = SentenceTransformer(model_name, device='cpu')
    outs = []
    for i in range(0, len(texts), batch_size):
        v = model.encode(texts[i:i+batch_size], batch_size=batch_size, show_progress_bar=False, convert_to_numpy=True, normalize_embeddings=True)
        outs.append(v)
    return np.vstack(outs)

cache_path = './data/embeddings_agnews.npz'
if os.path.exists(cache_path):
    dat = np.load(cache_path)
    E = dat['E']; idx_text = dat['idx_text']
    idx_text = idx_text.tolist()
else:
    _t0 = time.perf_counter()
    E = embed_corpus(corpus['text'].tolist(), batch_size=64)
    dt = time.perf_counter()-_t0
    print('임베딩(sec):', round(dt,2))
    idx_text = corpus['text'].tolist()
    np.savez_compressed(cache_path, E=E, idx_text=np.array(idx_text, dtype=object))

E.shape


### 3. 최근접 탐색(코사인) 및 검색 함수


In [ ]:
# sklearn NearestNeighbors로 코사인 기반 kNN 인덱스
nn = NearestNeighbors(metric='cosine', algorithm='brute')
nn.fit(E)


def search_topk(query: str, k: int = 5) -> list[tuple[int, float, str]]:
    # 쿼리 임베딩
    qv = embed_corpus([query], batch_size=1)
    dist, idx = nn.kneighbors(qv, n_neighbors=k, return_distance=True)
    # cosine distance → similarity = 1 - distance
    out = []
    for d, i in zip(dist[0], idx[0]):
        out.append((int(i), float(1.0 - d), idx_text[i]))
    return out

# 예시 검색
for i, (idx_i, sim, text) in enumerate(search_topk('New AI system beats benchmarks in vision tasks', k=5), 1):
    print(i, round(sim,3), text[:120]+'...')


### 4. 미니 RAG 조합 함수
- retrieved 문서들을 템플릿으로 단순 요약/응답 생성


In [ ]:
def compose_response(query: str, topk: list[tuple[int, float, str]], k: int = 3) -> str:
    parts = [f"Q: {query}", "\n[관련 문서 요약]"]
    for j, (idx_i, sim, text) in enumerate(topk[:k], 1):
        parts.append(f"- ({j}) 유사도 {sim:.2f}: {text[:160]}...")
    parts.append("\nA: 위 문서들을 참고하면, 질문과 직접적으로 연관된 핵심은 위 목록에서 확인 가능합니다.")
    return "\n".join(parts)

q = 'What is the impact of interest rate hikes on the stock market?'
res = search_topk(q, k=5)
print(compose_response(q, res, k=3))


### 5. 간단 데모
- 질의 입력 → top‑k 리스트와 응답 출력


In [ ]:
queries = [
    'Latest advances in computer vision benchmarks',
    'Effects of monetary policy on markets',
    'Championship highlights of the season',
]
for q in queries:
    res = search_topk(q, k=5)
    print('='*60)
    print(compose_response(q, res, k=3))
